# VGG Transfer Learning for Solar Flare Prediction

jupyter notebook to perform transfer learning with the VGG16 architecture.

Comments throughout the notebook indicate paths and other paramters that can be specified.

Displays performance within the notebook and outputs an hdf5 model file with the trained model for each epoch (can be configured to only output the final or best model with appropriate options in the tensorflow `model.fit` call.

Requires the the dataframes files as output by Build_dataframes.py and the SDO HMI AR Images corresponding to the dataframes.
  - The dataframes for the preconfigured reduced resolution dataset `Test_Data_by_AR_png_224.csv`, `Train_Data_by_AR_png_224.csv`, and `Validation_Data_by_AR_png_224.csv` are available on Dryad at `<insert link here>` and for the full resolution dataset `Test_Data_by_AR.csv`, `Train_Data_by_AR.csv`, and `Validation_Data_by_AR.csv` are available on Dryad at `insert link here>`. It is recommended that you save the dataframes in the `classifier_VGG/` directory (i.e., the same directory as the VGG code), although subsequent code will allow you to specify the path to those files.
 - The SDO HMI AR Images are available on Dryad at `<insert link here>` (reduced resolution png files) or `<insert link here>` (full resolution fits files)). The location of the SDO HMI AR Images will be specified in subsequent code. You may save those data in the base `AR-flares/` directory or any other location.

This code has only been tested on a GPU using tensorflow, but should work and/or be adaptable to CPU implementation (although the computation time would be excessive for CPU implementation).

References:
[1] L. E. Boucheron, T. Vincent, J. A. Grajeda, and E. Wuest, "Solar Active Region Magnetogram Image Dataset for Studies of Space Weather," arXiv preprint arXiv:2305.09492, 2023.

Copyright 2022 Laura Boucheron, Ty Vincent
This file is part of AR-flares

AR-flares is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AR-flares is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AR-flares. If not, see <https://www.gnu.org/licenses/>.


In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
import skimage.transform

import tensorflow.keras as keras

2025-03-21 13:25:47.756337: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available:", gpus)
else:
    print("No GPUs found.")


No GPUs found.


The following code cell can avoid cuDNN errors resulting from running out of GPU memory by allocating only what is needed at any given time.  This may be needed, especially if you are sharing your GPU with other processes (e.g., running your monitor).

In [3]:
# # This is to avoid CuDNN errors resulting from running out of GPU memory
# # This allocates only what is needed at any given time
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

Set the following flag to designate whether you are working with `.png` files or `.fits` files.  This flag will be used to use the correct dataloaders for each file type.  Note that the use of `.fits` files requires the use of a specialized dataloader (included below).

In [3]:
image_type = 'png' # 'png' or 'fits'

Instantiate the VGG16 architecture pretrained on imagenet.

In [4]:
model1 = keras.applications.vgg16.VGG16(include_top=True,weights='imagenet')

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 98s 0us/step


Replace the output layer with a 2-class softmax layer.

In [5]:
model1 = keras.models.Model(inputs=model1.input,outputs=model1.layers[-2].output) # amputate last dense layer
new_output = model1.output # take the output as currently defined
new_output = keras.layers.Dense(2,activation='softmax')(new_output) # operate on that output with another dense layer
model1 = keras.models.Model(inputs=model1.input,outputs=new_output) # define a new model with the new output

Freeze all layers except the last layer.

In [6]:
for layer in model1.layers[:-1]:
    layer.trainable=False

## Custom Data Generator for fits files

The following class defines a custom data generator for fits files.  

Approach taken from https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

From https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence: "Sequence are a safer way to do multiprocessing. This structure guarantees that the network will only train once on each sample per epoch which is not the case with generators."

In [7]:
class FitsDataGen(keras.utils.Sequence):
    # The input to the data generator will be the dataframe and which columns to use
    def __init__(self, df, X_col, y_col,
                 directory,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy() # dataframe
        self.X_col = X_col # column for X data (filename)
        self.y_col = y_col # column for y data (class label)
        self.directory = directory # base directory for data
        self.batch_size = batch_size # batch size
        self.input_size = input_size # size expected by network (224,224,3) for VGG
        self.shuffle = shuffle # whether to shuffle batches
        
        self.n = len(self.df) # number of data points
        self.nclasses = df[y_col].nunique() # number of classes
            
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, directory, input_size):
    
        with fits.open(directory+path) as img: # read in fits image
            img.verify('silentfix')
            img = img[1].data
            
        img = np.expand_dims(img,axis=2) # copy single channel to three to create rgb dimensioned image
        img = np.tile(img,(1,1,3))
        
        # scale to input_size (expected dimensions for input to network)
        img = skimage.transform.resize(img, (input_size[0],input_size[1]), order=1, mode='reflect',\
                                       clip=True, preserve_range=True, anti_aliasing=True)
        
        # scale intensities to range [0,255] as expected by VGG preprocessing function
        # can cheat a bit here and treat each channel the same since these are grayscale images
        img = img + 5978.7 # -5978.7 is minimum of entire magnetogram dataset
        img = img/(2*5978.7)*255 # +5978.7 is maximum of entire magnetogram dataset        
        
        img = keras.applications.vgg16.preprocess_input(img) # preprocess according to VGG expectations

        return img
    
    def __get_output(self, label, num_classes):
        return keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col]
        
        label_batch = batches[self.y_col]

        X_batch = np.asarray([self.__get_input(x, self.directory, self.input_size) for x in path_batch])

        y_batch = np.asarray([self.__get_output(y, self.nclasses) for y in label_batch])
        
        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

Read in dataframe specifying file locations for train, val, test.  Change these locations as needed.

In [9]:
if image_type=='fits':
    train_df = pd.read_csv('Train_Data_by_AR.csv',dtype=str)
    val_df = pd.read_csv('Validation_Data_by_AR.csv',dtype=str)
    test_df = pd.read_csv('Test_Data_by_AR.csv',dtype=str)

Create data generators using custom class defined above.  Change the `directory` parameter to point to the base directory for the fits files.  It is assumed that the directory structure underneath `directory` is of the form `XXXX/` where `XXXX` are four digit AR region numbers and that each `XXXX/` directory contains all `.fits` files associated with that AR.

In [10]:
if image_type=='fits':
    train_generator = FitsDataGen(train_df, X_col='filename', y_col='class',\
                                  directory='/mnt/solar_flares/AR_Dataset/Lat60_Lon60_Nans0/',\
                                  batch_size=64, input_size=(224,224,3), shuffle=True)
    val_generator = FitsDataGen(val_df, X_col='filename', y_col='class',\
                                directory='/mnt/solar_flares/AR_Dataset/Lat60_Lon60_Nans0/',\
                                batch_size=64, input_size=(224,224,3), shuffle=True)
    test_generator = FitsDataGen(test_df, X_col='filename', y_col='class',\
                                directory='/mnt/solar_flares/AR_Dataset/Lat60_Lon60_Nans0/',\
                                batch_size=64, input_size=(224,224,3), shuffle=True)

## Dataframe approach for png files

The following cells implement a data generator for `png` files using the native `ImageDataGenerator` class in `tensorflow.keras`.

Read in dataframe specifying file locations for train, val, test.  Change these locations as needed.

In [12]:
if image_type=='png':
    train_df = pd.read_csv('/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/cnn_features/Train_Data_by_AR_png_224.csv',dtype=str)
    val_df = pd.read_csv('/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/cnn_features/Validation_Data_by_AR_png_224.csv',dtype=str)
    test_df = pd.read_csv('/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/cnn_features/Test_Data_by_AR_png_224.csv',dtype=str)

Create data generators using the `ImageDataGenerator` class. Change the directory parameter to point to the base directory for the png files. It is assumed that the directory structure underneath directory is of the form `XXXX/` where `XXXX` are four digit AR region numbers and that each `XXXX/` directory contains all .png files associated with that AR.

Note--this code cell can take a long time to run the first time.

In [15]:
if image_type=='png':
    train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input)
    train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,\
                                                        directory='/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/Lat60_Lon60_Nans0_png_224',\
                                                        xcol='filename',y_col='class',\
                                                        target_size=(224,224), color_mode='rgb',\
                                                        batch_size=64, class_mode='categorical',\
                                                        shuffle=True)
    val_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input)
    val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,\
                                                    directory = '/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/Lat60_Lon60_Nans0_png_224',\
                                                    xcol='filename',ycol='class',\
                                                    target_size=(224,224), color_mode='rgb',\
                                                        batch_size=64, class_mode='categorical',\
                                                shuffle=True)
    test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input)
    test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,\
                                                    directory = '/Users/danilgarmaev/Documents/Masters_Research/AR-flares/data/Lat60_Lon60_Nans0_png_224',\
                                                    xcol='filename',ycol='class',\
                                                    target_size=(224,224), color_mode='rgb',\
                                                    batch_size=64, class_mode='categorical',\
                                                    shuffle=True)

Found 759357 validated image filenames belonging to 2 classes.
Found 95933 validated image filenames belonging to 2 classes.
Found 94757 validated image filenames belonging to 2 classes.


## Training Setup

Define custom metrics for true positive rate (TPR), true negative rate (TNR), Heidke Skill Score (HSS), and True Skill Statistic (TSS).  It is necessary to define these custom metrics to appropriately report batch to batch.  Since they are not linear metrics, you cannot simply average the batch reported metric.

Adapted from section "Creating Custom Metrics" at https://keras.io/api/metrics/

In [16]:
# custom metrics since these will not average batch to batch
class TNR(keras.metrics.Metric):
    def __init__(self, name='TNR', **kwargs):
        super(TNR, self).__init__(name=name, **kwargs)
        self.TN = self.add_weight(name='TN', initializer='zeros')
        self.FP = self.add_weight(name='FP', initializer='zeros')
        self.TNR = self.add_weight(name='TNR', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_true = 1 - y_true
        neg_y_pred = 1 - y_pred
        fp = keras.backend.cast(keras.backend.sum(neg_y_true * y_pred),'float32')
        tn = keras.backend.cast(keras.backend.sum(neg_y_true * neg_y_pred),'float32')
        
        self.TN.assign_add(tn)
        self.FP.assign_add(fp)
        
        tnr = self.TN / (self.TN + self.FP + keras.backend.epsilon())
        
        self.TNR.assign(tnr)

    def result(self):
        return self.TNR

    def reset_states(self):
        self.TN.assign(0)
        self.FP.assign(0)
        self.TNR.assign(0)
        
class TPR(keras.metrics.Metric):
    def __init__(self, name='TPR', **kwargs):
        super(TPR, self).__init__(name=name, **kwargs)
        self.TP = self.add_weight(name='TP', initializer='zeros')
        self.FN = self.add_weight(name='FN', initializer='zeros')
        self.TPR = self.add_weight(name='TPR', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_pred = 1 - y_pred
        fn = keras.backend.cast(keras.backend.sum(y_true * neg_y_pred),'float32')
        tp = keras.backend.cast(keras.backend.sum(y_true * y_pred),'float32')
        
        self.TP.assign_add(tp)
        self.FN.assign_add(fn)
        
        tpr = self.TP / (self.TP + self.FN + keras.backend.epsilon())
        
        self.TPR.assign(tpr)

    def result(self):
        return self.TPR

    def reset_states(self):
        self.TP.assign(0)
        self.FN.assign(0)
        self.TPR.assign(0)
        
class TSS(keras.metrics.Metric):
    def __init__(self, name='TSS', **kwargs):
        super(TSS, self).__init__(name=name, **kwargs)
        self.TP = self.add_weight(name='TP', initializer='zeros')
        self.TN = self.add_weight(name='TN', initializer='zeros')
        self.FP = self.add_weight(name='FP', initializer='zeros')
        self.FN = self.add_weight(name='FN', initializer='zeros')
        self.TSS = self.add_weight(name='TSS', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_true = 1 - y_true
        neg_y_pred = 1 - y_pred
        fp = keras.backend.cast(keras.backend.sum(neg_y_true * y_pred),'float32')
        tn = keras.backend.cast(keras.backend.sum(neg_y_true * neg_y_pred),'float32')
        fn = keras.backend.cast(keras.backend.sum(y_true * neg_y_pred),'float32')
        tp = keras.backend.cast(keras.backend.sum(y_true * y_pred),'float32')
        
        self.TP.assign_add(tp)
        self.TN.assign_add(tn)
        self.FP.assign_add(fp)
        self.FN.assign_add(fn)
        
        tnr = self.TN / (self.TN + self.FP + keras.backend.epsilon())
        tpr = self.TP / (self.TP + self.FN + keras.backend.epsilon())
        tss = tpr + tnr - 1
       
        self.TSS.assign(tss)

    def result(self):
        return self.TSS

    def reset_states(self):
        self.TP.assign(0)
        self.TN.assign(0)
        self.FP.assign(0)
        self.FN.assign(0)
        self.TSS.assign(0)
        
class HSS(keras.metrics.Metric):
    def __init__(self, name='HSS', **kwargs):
        super(HSS, self).__init__(name=name, **kwargs)
        self.TP = self.add_weight(name='TP', initializer='zeros')
        self.TN = self.add_weight(name='TN', initializer='zeros')
        self.FP = self.add_weight(name='FP', initializer='zeros')
        self.FN = self.add_weight(name='FN', initializer='zeros')
        self.HSS = self.add_weight(name='HSS', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_true = 1 - y_true
        neg_y_pred = 1 - y_pred
        fp = keras.backend.cast(keras.backend.sum(neg_y_true * y_pred),'float32')
        tn = keras.backend.cast(keras.backend.sum(neg_y_true * neg_y_pred),'float32')
        fn = keras.backend.cast(keras.backend.sum(y_true * neg_y_pred),'float32')
        tp = keras.backend.cast(keras.backend.sum(y_true * y_pred),'float32')
        
        self.TP.assign_add(tp)
        self.TN.assign_add(tn)
        self.FP.assign_add(fp)
        self.FN.assign_add(fn)
        
        hss = 2*(self.TP*self.TN-self.FN*self.FP)/((self.TP+self.FN)*(self.FN+self.TN)+(self.TP+self.FP)*(self.FP+self.TN))
       
        self.HSS.assign(hss)

    def result(self):
        return self.HSS

    def reset_states(self):
        self.TP.assign(0)
        self.TN.assign(0)
        self.FP.assign(0)
        self.FN.assign(0)
        self.HSS.assign(0)
        
class TN(keras.metrics.Metric):
    def __init__(self, name='TN', **kwargs):
        super(TN, self).__init__(name=name, **kwargs)
        self.TN = self.add_weight(name='TN', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_true = 1 - y_true
        neg_y_pred = 1 - y_pred
        tn = keras.backend.cast(keras.backend.sum(neg_y_true * neg_y_pred),'float32')

        self.TN.assign_add(tn)

    def result(self):
        return self.TN

    def reset_states(self):
        self.TN.assign(0)
        
class TP(keras.metrics.Metric):
    def __init__(self, name='TP', **kwargs):
        super(TP, self).__init__(name=name, **kwargs)
        self.TP = self.add_weight(name='FP', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        tp = keras.backend.cast(keras.backend.sum(y_true * y_pred),'float32')

        self.TP.assign_add(tp)

    def result(self):
        return self.TP

    def reset_states(self):
        self.TP.assign(0)
        
class FN(keras.metrics.Metric):
    def __init__(self, name='FN', **kwargs):
        super(FN, self).__init__(name=name, **kwargs)
        self.FN = self.add_weight(name='FN', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_pred = 1 - y_pred
        fn = keras.backend.cast(keras.backend.sum(y_true * neg_y_pred),'float32')

        self.FN.assign_add(fn)

    def result(self):
        return self.FN

    def reset_states(self):
        self.FN.assign(0)
        
class FP(keras.metrics.Metric):
    def __init__(self, name='FP', **kwargs):
        super(FP, self).__init__(name=name, **kwargs)
        self.FP = self.add_weight(name='FP', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = keras.backend.argmax(y_true)
        y_pred = keras.backend.argmax(y_pred)
        neg_y_true = 1 - y_true
        fp = keras.backend.cast(keras.backend.sum(neg_y_true * y_pred),'float32')

        self.FP.assign_add(fp)

    def result(self):
        return self.FP

    def reset_states(self):
        self.FP.assign(0)

Define optimizer.

In [17]:
adam_opt = keras.optimizers.Adam(learning_rate=0.001)
model1.compile(loss='categorical_crossentropy', optimizer=adam_opt,\
               metrics=[TNR(), TPR(), TSS()])

Set up remaining training parameters.  Set `results_dir` to desired output directory.  This is where models will be saved each epoch.  To save only the best model, change the `save_best_only` parameter in the `checkpoint` variable to `True`.  The classes are weighted such that non-flares (majority) are weighted 1 and flares (minority) are weighted as $N_n/N_f$ where $N_n$ is the number of non-flaring examples and $N_f$ is the number of flaring examples.

In [21]:
results_dir = 'png'

filepath = 'models/'+results_dir+'/model.{epoch:02d}_{val_TSS:.2f}.hdf5.keras'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_TSS', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

step_size_train = int(np.ceil(train_generator.n/train_generator.batch_size) )
step_size_val = int(np.ceil(val_generator.n/val_generator.batch_size))

# the following assumes that 0 is the majority class
class_weights = {0:1., 1: (train_df['class']=='0').sum()/(train_df['class']=='1').sum()}

## Training

Especially for the fits dataset, the file I/O will likely be a significant bottleneck.  Code is provided below for both single-threaded implementation and multi-threaded implementation.  The multi-threaded implementation can case nondeterministic deadlocks (as will likely be reported as a warning by tensorflow), but can significantly speed up training.

### Single threaded

In [22]:
history = model1.fit(train_generator, steps_per_epoch=step_size_train, epochs=5, verbose=1,\
                     callbacks=callbacks_list, validation_data=val_generator, validation_steps=step_size_val,\
                     validation_freq=1, class_weight=class_weights)

Epoch 1/5


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


    6/11865 ━━━━━━━━━━━━━━━━━━━━ 122:38:14 37s/step - TNR: 0.5811 - TPR: 0.5387 - TSS: 0.1198 - loss: 1.5443 

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x1367cede0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


KeyboardInterrupt: 

### Multi-threaded
This code can cause issues if you try to cancel the code after it has begun running.  You may have to manually kill python processes associated with the workers in order to get the code to stop running.  You may need to adjust the `max_queue_size` and `workers` according to available CPU computational horsepower.

In [23]:
history = model1.fit(train_generator, steps_per_epoch=step_size_train, epochs=5, verbose=1,\
                     callbacks=callbacks_list, validation_data=val_generator, validation_steps=step_size_val,\
                     validation_freq=1, class_weight=class_weights,\
                    max_queue_size=100, workers=20, use_multiprocessing=True)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'max_queue_size'

## Evaluate

The following code will evaluate the transfer learned model on the test dataset.  Change the variable `model_file` to point to the `hdf5` file that you wish to evaluate the performance of.

In [25]:
model_file = 'models/png/model.02_0.48.hdf5'
model1 = keras.models.load_model(model_file, custom_objects={'TNR': TNR(),\
                                                       'TPR': TPR(),\
                                                       'TSS': TSS(),\
                                                       'HSS': HSS()})
model1.compile(loss='categorical_crossentropy', optimizer=adam_opt,\
               metrics=[TNR(), TPR(), TSS(), HSS()])

### Single threaded

In [27]:
model1.evaluate(test_generator)

1481/1481 [==============================] - 165s 111ms/step - loss: 0.5018 - TNR: 0.7980 - TPR: 0.7345 - TSS: 0.5325 - HSS: 0.4636


[0.5018448829650879,
 0.7980331778526306,
 0.7344722151756287,
 0.5325053930282593,
 0.4636412262916565]

### Multi threaded
This code can cause issues if you try to cancel the code after it has begun running.  You may have to manually kill python processes associated with the workers in order to get the code to stop running.  You may need to adjust the `max_queue_size` and `workers` according to available CPU computational horsepower.

In [26]:
model1.evaluate(test_generator,max_queue_size=100, workers=20, use_multiprocessing=True)

1481/1481 [==============================] - 178s 119ms/step - loss: 0.5018 - TNR: 0.7980 - TPR: 0.7345 - TSS: 0.5325 - HSS: 0.4636


[0.5018448829650879,
 0.7980331778526306,
 0.7344722151756287,
 0.5325053930282593,
 0.4636412262916565]